# Init & Login

In [1]:
from math import ceil
from random import randrange
from time import sleep
from pprint import pprint

import spotipy
import spotipy.util as util
from IPython.display import clear_output

## Client Info ##
CLIENT_ID     = ""
CLIENT_SECRET = ""
CLIENT_SCOPE  = "user-follow-modify playlist-modify-private playlist-modify-public"
USER_NAME     = "31ytgsr7wdmiaroy77msqpiupdsi"
REDIR_URI     = "https://github.com/jwatson-CO-edu/yt_shuffle_so_good"
AUTH_URL      = 'https://accounts.spotify.com/api/token'
BASE_URL      = 'https://api.spotify.com/v1/'
## API Info ##
_RESPONSE_LIMIT = 100

with open( "../keys/spot_ID.txt" , 'r' ) as f:
    CLIENT_ID = f.readlines()[0].strip()

with open( "../keys/spot_SECRET.txt" , 'r' ) as f:
    CLIENT_SECRET = f.readlines()[0].strip()

token = None
token = util.prompt_for_user_token(
    username      = USER_NAME,
    scope         = CLIENT_SCOPE,
    client_id     = CLIENT_ID,
    client_secret = CLIENT_SECRET,
    redirect_uri  = REDIR_URI
)

print( token )

spot = spotipy.Spotify( auth = token )
clear_output( wait = True )
sleep( 5 )
print( "TOKEN OBTAINED" )

TOKEN OBTAINED


# Playlists

In [2]:

playlist = {
    'study01' : "0a2qoe6S7lYeZ6nlhZdA0v",
    'study02' : "6gbtR2cBq5PvkghidCvvGk",
    'study03' : "3o3lN2qntdEV7UKTuuC77K",
    'study04' : "41sFSisljvBDMBXtpp5NIw",
    'study05' : "02iS5AFGp8YVuUUqcQf8ys",
    'study06' : "6KI7A4MWrSM7EyKRUjxIi1",
    'study07' : "3V055Md2JdrUT8tX0af7di",
    'study08' : "0tspdJlwSgiyf2O9PO6QaP",
    'study09' : "5mHRBFoQtYy2izeZ66pG95",
    'study10' : "3832xeKGEOAXFJqE4K8kIq",
    'study11' : "65MXR4dubPL9t0P4dgTWvn",
    'study12' : "0ecSAfnD4CulIVnLt26ukI",
    'study13' : "7K9ucByFRgDuZk8KMHeJkL",
    'zd_Over' : "0v26bHydUxcGC5EbMlkjzG",
}

dupeDump = "1VPXM7m1by79EdEzDqGsHy"


# Playlist Functions

In [3]:

def fetch_entire_playlist( playlist_ID ):
    """ Get infodump on all plalist tracks """
    plTracks = []
    trCount  = 0
    response = spot.user_playlist_tracks(
        CLIENT_ID, 
        playlist_ID, 
        fields = 'items,uri,name,id,total', 
        limit  = _RESPONSE_LIMIT
    )
    Ntracks = response['total']
    while 1:
        trCount += len(response['items'])
        plTracks.extend( response['items'] )
        
        if trCount >= Ntracks:
            break
    
        response = spot.user_playlist_tracks(
            CLIENT_ID, 
            playlist_ID, 
            fields = 'items,uri,name,id,total', 
            limit  = _RESPONSE_LIMIT,
            offset = trCount
        )
    return plTracks


def get_playlist_unique_ID_set( plItems ):
    """ Get all IDs from the playlist without repeats """
    uniqID = set([])
    for item in plItems:
        uniqID.add( item['track']['id'] )
    print( f"Playlist of {len(plItems)} has {len(uniqID)} unique tracks!" )
    return uniqID 

def get_playlist_series_unique_ID_set( plDict ):
    """ Get all IDs from each playlist without repeats """
    
    print( f"##### First Pass #####\n" )
    bgnSets = []
    nameLst = []
    idList  = []
    for plName, plID in plDict.items():
        print( f"### {plName} ###" )
        nameLst.append( plName )
        idList.append( plID )
        bgnSets.append( get_playlist_unique_ID_set( fetch_entire_playlist( plID ) ) )
        print()

    print( f"##### Second Pass #####\n" )
    endSets = [ bgnSets[0], ]
    print( len( endSets[0] ), end = ", ", flush = True )
    for trackSet in bgnSets[1:]:
        nuSet = set([])
        for elem in trackSet:
            found = False
            for compSet in endSets:
                if elem in compSet:
                    found = True
                    break
            if not found:
                nuSet.add( elem )
        print( len( nuSet ), end = ", ", flush = True )
        if len( nuSet ):
            endSets.append( nuSet )
    print( '\n' )
    return zip( nameLst, idList, endSets )


def create_dupe_removal_jobs( plDict, pause_s = 0.5 ):
    """ Remove all duplicates from each playlist while attempting to preserve them in case of a massive fuckup """    
    uniqList = list()
    srtdKeys = sorted( list( plDict.keys() ) )
    dumpList = list()
    plIDlist = list()

    def p_exists_in_prev( itemID, currDex ):
        """ Return True if `itemID` exists in a playlist before `currDex`, Otherwise return False """
        for plSet in uniqList[ :currDex ]:
            if itemID in plSet:
                return True
        return False

    # 1. For every playlist in the dict, do
    for i, plName in enumerate( srtdKeys ):

        print( f"##### Playlist {i+1}: {plName} #####" )
        
        # 2. Fetch playlist and establish a running set
        plID      = plDict[ plName ]
        trkList_i = fetch_entire_playlist( plID )
        trkSet_i  = set([])
        dumpLst_i = list()
        plIDlist.append( plID )
        print( f"Fetched {len(trkList_i)} tracks!" )
        
        # 3. For every track j in playlist i, do
        for j, track_j in enumerate( trkList_i ):

            p_dump_j = False
            
            # 4. Test 1: Did we find this song earlier in the playlist?
            trackID_j = track_j['track']['id']
            if trackID_j in trkSet_i:
                dumpLst_i.append( trackID_j )
                p_dump_j = True

            # 5. Test 2: Did we find this song in and earlier playlist?
            if (not p_dump_j) and p_exists_in_prev( trackID_j, i ):
                dumpLst_i.append( trackID_j )
                p_dump_j = True
            
            # 6. Uniqify
            if not p_dump_j:
                trkSet_i.add( trackID_j )
            
        # 7. Store track set i and dump list
        uniqList.append( trkSet_i )
        dumpList.append( dumpLst_i )
        print( f"In {plName}/{plID}: Retain {len(trkSet_i)}, Dump {len(dumpLst_i)}, Valid? {(len(trkSet_i)+len(dumpLst_i))==len(trkList_i)}\n" )
        sleep( pause_s )

    # N. Return removal job list
    return list( zip( plIDlist, dumpList ) )
            

def run_dupe_removal_jobs( jobList, plTarget, pause_s = 0.5 ):
    """ Run jobs created in `create_dupe_removal_jobs` and store them in `plTarget` in the event of a massive fuckup """
    print( f"########## About to run {len(jobList)} jobs ... ##########\n" )
    for i, (plID_i, remLst_i) in enumerate( jobList ):
        
        print( f"##### Job {i+1}: {plID_i}, {len(remLst_i)} to remove ... #####" )
        
        print( f"Venting dupes to {plTarget} ..." )
        spot.user_playlist_add_tracks( CLIENT_ID, plTarget, remLst_i )
        sleep( pause_s )
        
        print( f"removing dupes from {plID_i} ..." )
        spot.playlist_remove_all_occurrences_of_items( plID_i, remLst_i )
        sleep( pause_s )
        
        print()
    print( f"########## Completed {len(jobList)} jobs! ##########\n" )
        

# Identify Duplicates

In [4]:
jobList = create_dupe_removal_jobs( playlist )

##### Playlist 1: study01 #####
Fetched 400 tracks!
In study01/0a2qoe6S7lYeZ6nlhZdA0v: Retain 397, Dump 3, Valid? True

##### Playlist 2: study02 #####
Fetched 400 tracks!
In study02/6gbtR2cBq5PvkghidCvvGk: Retain 392, Dump 8, Valid? True

##### Playlist 3: study03 #####
Fetched 400 tracks!
In study03/3o3lN2qntdEV7UKTuuC77K: Retain 359, Dump 41, Valid? True

##### Playlist 4: study04 #####
Fetched 400 tracks!
In study04/41sFSisljvBDMBXtpp5NIw: Retain 374, Dump 26, Valid? True

##### Playlist 5: study05 #####
Fetched 400 tracks!
In study05/02iS5AFGp8YVuUUqcQf8ys: Retain 373, Dump 27, Valid? True

##### Playlist 6: study06 #####
Fetched 400 tracks!
In study06/6KI7A4MWrSM7EyKRUjxIi1: Retain 375, Dump 25, Valid? True

##### Playlist 7: study07 #####
Fetched 400 tracks!
In study07/3V055Md2JdrUT8tX0af7di: Retain 366, Dump 34, Valid? True

##### Playlist 8: study08 #####
Fetched 464 tracks!
In study08/0tspdJlwSgiyf2O9PO6QaP: Retain 436, Dump 28, Valid? True

##### Playlist 9: study09 #####
Fe

# Move Duplicates

In [5]:
run_dupe_removal_jobs( jobList, dupeDump, pause_s = 0.5 )

########## About to run 15 jobs ... ##########

##### Job 1: 0a2qoe6S7lYeZ6nlhZdA0v, 3 to remove ... #####
Venting dupes to 1VPXM7m1by79EdEzDqGsHy ...
removing dupes from 0a2qoe6S7lYeZ6nlhZdA0v ...

##### Job 2: 6gbtR2cBq5PvkghidCvvGk, 8 to remove ... #####
Venting dupes to 1VPXM7m1by79EdEzDqGsHy ...
removing dupes from 6gbtR2cBq5PvkghidCvvGk ...

##### Job 3: 3o3lN2qntdEV7UKTuuC77K, 41 to remove ... #####
Venting dupes to 1VPXM7m1by79EdEzDqGsHy ...
removing dupes from 3o3lN2qntdEV7UKTuuC77K ...

##### Job 4: 41sFSisljvBDMBXtpp5NIw, 26 to remove ... #####
Venting dupes to 1VPXM7m1by79EdEzDqGsHy ...
removing dupes from 41sFSisljvBDMBXtpp5NIw ...

##### Job 5: 02iS5AFGp8YVuUUqcQf8ys, 27 to remove ... #####
Venting dupes to 1VPXM7m1by79EdEzDqGsHy ...
removing dupes from 02iS5AFGp8YVuUUqcQf8ys ...

##### Job 6: 6KI7A4MWrSM7EyKRUjxIi1, 25 to remove ... #####
Venting dupes to 1VPXM7m1by79EdEzDqGsHy ...
removing dupes from 6KI7A4MWrSM7EyKRUjxIi1 ...

##### Job 7: 3V055Md2JdrUT8tX0af7di, 34 to